# Garmin Fenix 7S

<p align="left">
  <img src="https://res.garmin.com/transform/image/upload/b_rgb:FFFFFF,c_pad,dpr_2.0,f_auto,h_400,q_auto,w_400/c_pad,h_400,w_400/v1/Product_Images/en/products/010-02539-00/v/cf-xl?pgw=1" width="300"/>
</p>

A versatile watch that is capable of tracking your heart rate, steps, and other metrics, the [Garmin Fenix 7S](https://www.garmin.com/en-US/p/735542) is a watch that stores a treasure trove of health data that we will explore in this notebook.

We've used the Garmin Fenix 7S for a while now, and we'll show you how to extract its data, visualize data, and compute correlations and other statistical measures based on your data! While you will need a Garmin Fenix 7S to actually collect the data itself, the data *extraction* requires only an internet connection and your username and password.

If you want to know more about the Garmin Fenix 7S, see the [Wearipedia](https://www.wearipedia.com/wearables/garmin-fenix) for a detailed analysis of performances, sensors, data privacy, and extraction pipelines.

You can extract the parameters as below:

| Parameter Name                  | Input Parameter                  | Frequency          | Return Type |
|---------------------------------|----------------------------------|---------------------|-------------|
| Stats                           | "stats"                          | 1 Day               | List        |
| Body Composition                | "body_composition"               | Entire Period      | JSON        |
| Body Composition Aggregated     | "body_composition_aggregated"    | 1 Day               | List        |
| Stats and Body Aggregated       | "stats_and_body_aggregated"      | 1 Day               | List        |
| Steps                           | "steps"                          | 1 Day (15 Mins)     | List        |
| Daily Steps                     | "daily_steps"                    | 1 Day               | List        |
| Body Battery                    | "body_battery"                   | 1 Day               | List        |
| Heart Rate                      | "hr"                             | 1 Day               | List        |
| Training Readiness              | "training_readiness"             | 1 Day               | List        |
| Blood Pressure                  | "blood_pressure"                 | Entire Period      | JSON        |
| Floors                          | "floors"                         | 1 Day (15 Mins)     | List        |
| Training Status                 | "training_status"                | 1 Day               | List        |
| Resting Heart Rate              | "rhr"                            | Entire Period (1 Day) | JSON      |
| Hydration                       | "hydration"                      | 1 Day               | List        |
| Sleep                           | "sleep"                          | 1 Day               | List        |
| Stress                          | "stress"                         | 1 Day               | List        |
| Day Stress Aggregated           | "day_stress_aggregated"          | 1 Day               | List        |
| Respiration                     | "respiration"                    | 1 Day               | List        |
| Race Prediction                 | "race_prediction"                | None                | List        |
| SPO2                            | "spo2"                           | 1 Day               | List        |
| Max Metrics                     | "max_metrics"                    | 1 Day               | List        |
| Personal Records                | "personal_records"               | Random              | List        |
| Earned Badges                   | "earned_badges"                  | Random              | List        |
| Adhoc Challenges                | "adhoc_challenges"               | None                | List        |
| Available Badges                | "available_badges"               | Random              | List        |
| Available Badge Challenges      | "available_badge_challenges"     | Random              | List        |
| Badge Challenges                | "badge_challenges"               | None                | List        |
| Non Completed Badge Challenges  | "non_completed_badge_challenges" | None                | List        |
| Activities                      | "activities"                     | 1 Day (Random)      | List        |
| Activities Dated                | "activities_date"                | 1 Day (Random)      | List        |
| Activities Fordate Aggregated   | "activities_fordate_aggregated"  | 1 Day               | List        |
| Devices                         | "devices"                        | Constant (3)        | List        |
| Device Last Used                | "device_last_used"               | Constant (1)        | JSON        |
| Device Settings                 | "device_settings"                | Constant (3)        | List        |
| Device Alarms                   | "device_alarms"                  | None                | List        |
| Active Goals                    | "active_goals"                   | None                | List        |
| Future Goals                    | "future_goals"                   | None                | List        |
| Past Goals                      | "past_goals"                     | None                | List        |
| Heart Rate Variability          | "hrv"                            | 1 Day               | List        |
| Weigh Ins                       | "weigh_ins"                      | Entire Period      | JSON        |
| Daily Weigh Ins                 | "weigh_ins_daily"                | 1 Day               | List        |
| Hill Score                      | "hill_score"                     | Entire Period      | JSON        |
| Endurance Score                 | "endurance_score"                | Entire Period      | JSON        |


In this guide, we sequentially cover the following **nine** topics to extract from the unofficial Garmin API:
1. **Setup**
    - Participant Setup
    - Data Receiver Setup
    - We install the necessary packages and import them.
2. **Authentication/Authorization**
    - Requires only username and password. That's it!
3. **Data extraction**
    - We get data via `wearipedia` in a couple lines of code.
4. **Data Exporting**
    - We export all of this data to file formats compatible by R, Excel, and MatLab.
5. **Visualization**
    - We create a simple plot to visualize our data.
6. **Advanced visualization**
    - In this section, we delve into the realm of advanced visualization techniques. Once you've collected and processed your Garmin Fenix 7S data, you'll want to take your insights to the next level.
    - We'll explore cutting-edge data visualization methods that can help you gain deeper insights into your health and fitness journey. From interactive charts and graphs to 3D visualizations, we'll show you how to transform your data into compelling visuals that can uncover hidden patterns and trends.
    - Whether you're an athlete looking to optimize your training or simply curious about your health metrics, these advanced visualization techniques will empower you to make data-driven decisions with confidence. Let's dive in and unlock the full potential of your Garmin Fenix 7S data!
7. **Outlier Detection and Data Cleaning**
    - We detect outliers in our data and filter them out.
8. **Data analysis**

    - In this crucial phase of our data exploration journey, we shift our focus to extracting meaningful insights from the wealth of data collected from your Garmin Fenix 7S.
    - We'll employ a wide range of data analysis techniques to uncover patterns, correlations, and trends within your health and fitness data. From statistical analyses that provide a deeper understanding of your performance.
    - Whether you're interested in tracking your progress over time, identifying factors affecting your health, or simply gaining a better understanding of your data, this section will equip you with the tools and knowledge to conduct thorough data analysis.
    - Let's harness the power of data to enhance your well-being and achieve your fitness goals!
    

Disclaimer: this notebook is purely for educational purposes. All of the data currently stored in this notebook is purely *synthetic*, meaning randomly generated according to rules we created. Despite this, the end-to-end data extraction pipeline has been tested on our own data, meaning that if you enter your own email and password on your own Colab instance, you can visualize your own *real* data. That being said, we were unable to thoroughly test the timezone functionality, though, since we only have one account, so beware.

# 1. Setup

## Participant Setup

Dear Participant,

Once you unbox your Garmin Fenix 7S, please set up the device by following these resources:
- Written guide: https://support.garmin.com/en-US/?faq=jvJTLgrDV881K1NXuC4CsA
- Video guide: https://www.youtube.com/watch?v=Fgq6LM2NqNo

Make sure that your phone is paired to it using the Garmin Connect login credentials (email and password) given to you by the data receiver.

Best,

Wearipedia

## Data Receiver Setup

Please follow the below steps:

1. Create an email address for the participant, for example `foo@email.com`.
2. Create a Garmin Connect account with the email and some random password.
3. Keep email and password stored somewhere safe.
4. Distribute the device to the participant and instruct them to follow the participant setup letter above.
5. Install the `wearipedia` Python package to easily extract data from this device via the Garmin Connect API.

In [ ]:
%pip install https://github.com/HarshKhilawala/wearipedia/archive/add-data.zip

# 2. Authentication/Authorization

To obtain access to data, authorization is required. All you'll need to do here is just put in your email and password for your Garmin Fenix 7S device. We'll use this username and password to extract the data in the sections below.

In [ ]:
#@title Enter Garmin login credentials
email_address = "jadongeathers@gmail.com" #@param {type:"string"}
password = "StanfordGarminD4ta" #@param {type:"string"}

# 3. Data Extraction

Data can be extracted via [wearipedia](https://github.com/Stanford-Health/wearipedia/), our open-source Python package that unifies dozens of complex wearable device APIs into one simple, common interface.

First, we'll set a date range and then extract all of the data within that date range. You can select whether you would like synthetic data or not with the checkbox.

In [ ]:
#@title Enter start and end dates (in the format yyyy-mm-dd)

#set start and end dates - this will give you all the data from 2000-01-01 (January 1st, 2000) to 2100-02-03 (February 3rd, 2100), for example
start_date='2022-03-01' #@param {type:"string"}
end_date='2022-06-17' #@param {type:"string"}
synthetic = True #@param {type:"boolean"}

In [ ]:
import wearipedia

device = wearipedia.get_device("garmin/fenix_7s")

if not synthetic:
    device.authenticate({"email": email_address, "password": password})

params = {"start_date": start_date, "end_date": end_date}

In [ ]:
#@title Extracting all datafields
stats = device.get_data("stats", params=params)
body_composition = device.get_data("body_composition", params=params)
body_composition_aggregated = device.get_data("body_composition_aggregated", params=params)
stats_and_body_aggregated = device.get_data("stats_and_body_aggregated", params=params)
steps = device.get_data("steps", params=params)
daily_steps = device.get_data("daily_steps", params=params)
body_battery = device.get_data("body_battery", params=params)
hr = device.get_data("hr", params=params)
training_readiness = device.get_data("training_readiness", params=params)
blood_pressure = device.get_data("blood_pressure", params=params)
floors = device.get_data("floors", params=params)
training_status = device.get_data("training_status", params=params)
rhr = device.get_data("rhr", params=params)
hydration = device.get_data("hydration", params=params)
sleep = device.get_data("sleep", params=params)
stress = device.get_data("stress", params=params)
day_stress_aggregated = device.get_data("day_stress_aggregated", params=params)
respiration = device.get_data("respiration", params=params)
race_prediction = device.get_data("race_prediction", params=params)
spo2 = device.get_data("spo2", params=params)
max_metrics = device.get_data("max_metrics", params=params)
personal_record = device.get_data("personal_record", params=params)
earned_badges = device.get_data("earned_badges", params=params)
adhoc_challenges = device.get_data("adhoc_challenges", params=params)
available_badges = device.get_data("available_badges", params=params)
available_badge_challenges = device.get_data("available_badge_challenges", params=params)
badge_challenges = device.get_data("badge_challenges", params=params)
non_completed_badge_challenges = device.get_data("non_completed_badge_challenges", params=params)
activities = device.get_data("activities", params=params)
activities_date = device.get_data("activities_date", params=params)
activities_fordate_aggregated = device.get_data("activities_fordate_aggregated", params=params)
devices = device.get_data("devices", params=params)
device_last_used = device.get_data("device_last_used", params=params)
device_settings = device.get_data("device_settings", params=params)
device_alarms = device.get_data("device_alarms", params=params)
active_goals = device.get_data("active_goals", params=params)
future_goals = device.get_data("future_goals", params=params)
past_goals = device.get_data("past_goals", params=params)
hrv = device.get_data("hrv", params=params)
weigh_ins = device.get_data("weigh_ins", params=params)
weigh_ins_daily = device.get_data("weigh_ins_daily", params=params)
hill_score = device.get_data("hill_score", params=params)
endurance_score = device.get_data("endurance_score", params=params)
inprogress_virtual_challenges = device.get_data("inprogress_virtual_challenges", params=params)

# 4. Data Exporting

In this section, we export all of this data to formats compatible with popular scientific computing software (R, Excel, Google Sheets, Matlab). Specifically, we will first export to JSON, which can be read by R and Matlab. Then, we will export to CSV, which can be consumed by Excel, Google Sheets, and every other popular programming language.

## Exporting to JSON (R, Matlab, etc.)

Exporting to JSON is fairly simple. We export each datatype separately and also export a complete version that includes all simultaneously.

In [ ]:
import json

json.dump(stats, open("stats.json", "w"))
json.dump(body_composition, open("body_composition.json", "w"))
json.dump(body_composition_aggregated, open("body_composition_aggregated.json", "w"))
json.dump(stats_and_body_aggregated, open("stats_and_body_aggregated.json", "w"))
json.dump(steps, open("steps.json", "w"))
json.dump(daily_steps, open("daily_steps.json", "w"))
json.dump(body_battery, open("body_battery.json", "w"))
json.dump(hr, open("hr.json", "w"))
json.dump(training_readiness, open("training_readiness.json", "w"))
json.dump(blood_pressure, open("blood_pressure.json", "w"))
json.dump(floors, open("floors.json", "w"))
json.dump(training_status, open("training_status.json", "w"))
json.dump(rhr, open("rhr.json", "w"))
json.dump(hydration, open("hydration.json", "w"))
json.dump(sleep, open("sleep.json", "w"))
json.dump(stress, open("stress.json", "w"))
json.dump(day_stress_aggregated, open("day_stress_aggregated.json", "w"))
json.dump(respiration, open("respiration.json", "w"))
json.dump(race_prediction, open("race_prediction.json", "w"))
json.dump(spo2, open("spo2.json", "w"))
json.dump(max_metrics, open("max_metrics.json", "w"))
json.dump(personal_record, open("personal_record.json", "w"))
json.dump(earned_badges, open("earned_badges.json", "w"))
json.dump(adhoc_challenges, open("adhoc_challenges.json", "w"))
json.dump(available_badges, open("available_badges.json", "w"))
json.dump(available_badge_challenges, open("available_badge_challenges.json", "w"))
json.dump(badge_challenges, open("badge_challenges.json", "w"))
json.dump(non_completed_badge_challenges, open("non_completed_badge_challenges.json", "w"))
json.dump(activities, open("activities.json", "w"))
json.dump(activities_date, open("activities_date.json", "w"))
json.dump(activities_fordate_aggregated, open("activities_fordate_aggregated.json", "w"))
json.dump(devices, open("devices.json", "w"))
json.dump(device_last_used, open("device_last_used.json", "w"))
json.dump(device_settings, open("device_settings.json", "w"))
json.dump(device_alarms, open("device_alarms.json", "w"))
json.dump(active_goals, open("active_goals.json", "w"))
json.dump(future_goals, open("future_goals.json", "w"))
json.dump(past_goals, open("past_goals.json", "w"))
json.dump(hrv, open("hrv.json", "w"))
json.dump(weigh_ins, open("weigh_ins.json", "w"))
json.dump(weigh_ins_daily, open("weigh_ins_daily.json", "w"))
json.dump(hill_score, open("hill_score.json", "w"))
json.dump(endurance_score, open("endurance_score.json", "w"))
json.dump(inprogress_virtual_challenges, open("inprogress_virtual_challenges.json", "w"))

complete = {
    "stats": stats,
    "body_composition": body_composition,
    "body_composition_aggregated": body_composition_aggregated,
    "stats_and_body_aggregated": stats_and_body_aggregated,
    "steps": steps,
    "daily_steps": daily_steps,
    "body_battery": body_battery,
    "hr": hr,
    "training_readiness": training_readiness,
    "blood_pressure": blood_pressure,
    "floors": floors,
    "training_status": training_status,
    "rhr": rhr,
    "hydration": hydration,
    "sleep": sleep,
    "stress": stress,
    "day_stress_aggregated": day_stress_aggregated,
    "respiration": respiration,
    "race_prediction": race_prediction,
    "spo2": spo2,
    "max_metrics": max_metrics,
    "personal_record": personal_record,
    "earned_badges": earned_badges,
    "adhoc_challenges": adhoc_challenges,
    "available_badges": available_badges,
    "available_badge_challenges": available_badge_challenges,
    "badge_challenges": badge_challenges,
    "non_completed_badge_challenges": non_completed_badge_challenges,
    "activities": activities,
    "activities_date": activities_date,
    "activities_fordate_aggregated": activities_fordate_aggregated,
    "devices": devices,
    "device_last_used": device_last_used,
    "device_settings": device_settings,
    "device_alarms": device_alarms,
    "active_goals": active_goals,
    "future_goals": future_goals,
    "past_goals": past_goals,
    "hrv": hrv,
    "weigh_ins": weigh_ins,
    "weigh_ins_daily": weigh_ins_daily,
    "hill_score": hill_score,
    "endurance_score": endurance_score,
    "inprogress_virtual_challenges": inprogress_virtual_challenges
}

json.dump(complete, open("complete.json", "w"))

Feel free to open the file viewer (see left pane) to look at the outputs!

## Exporting to CSV and XLSX (Excel, Google Sheets, R, Matlab, etc.)

Exporting to CSV/XLSX requires a bit more processing, since they enforce a pretty restrictive schema.

We will thus export steps, heart rates, and breath rates all as separate files.

In [ ]:
import pandas as pd

# Stats
df = pd.DataFrame(stats)
df.to_csv('stats.csv', index=False)
df.to_excel('stats.xlsx', index=False)

# Body Composition
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([body_composition])
df.to_csv('body_composition.csv', index=False)
df.to_excel('body_composition.xlsx', index=False)

# Body Composition Aggregated
df = pd.DataFrame(body_composition_aggregated)
df.to_csv('body_composition_aggregated.csv', index=False)
df.to_excel('body_composition_aggregated.xlsx', index=False)

# Stats and Body Aggregated
df = pd.DataFrame(stats_and_body_aggregated)
df.to_csv('stats_and_body_aggregated.csv', index=False)
df.to_excel('stats_and_body_aggregated.xlsx', index=False)

# Steps
df = pd.DataFrame(steps)
df.to_csv('steps.csv', index=False)
df.to_excel('steps.xlsx', index=False)

# Daily Steps
df = pd.DataFrame(daily_steps)
df.to_csv('daily_steps.csv', index=False)
df.to_excel('daily_steps.xlsx', index=False)

# Body Battery
df = pd.DataFrame(body_battery)
df.to_csv('body_battery.csv', index=False)
df.to_excel('body_battery.xlsx', index=False)

# HR
df = pd.DataFrame(hr)
df.to_csv('hr.csv', index=False)
df.to_excel('hr.xlsx', index=False)

# Training Readiness
df = pd.DataFrame(training_readiness)
df.to_csv('training_readiness.csv', index=False)
df.to_excel('training_readiness.xlsx', index=False)

# Blood Pressure
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([blood_pressure])
df.to_csv('blood_pressure.csv', index=False)
df.to_excel('blood_pressure.xlsx', index=False)

# Floors
df = pd.DataFrame(floors)
df.to_csv('floors.csv', index=False)
df.to_excel('floors.xlsx', index=False)

# Training Status
df = pd.DataFrame(training_status)
df.to_csv('training_status.csv', index=False)
df.to_excel('training_status.xlsx', index=False)

# RHR
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([rhr])
df.to_csv('rhr.csv', index=False)
df.to_excel('rhr.xlsx', index=False)

# Hydration
df = pd.DataFrame(hydration)
df.to_csv('hydration.csv', index=False)
df.to_excel('hydration.xlsx', index=False)

# Sleep
df = pd.DataFrame(sleep)
df.to_csv('sleep.csv', index=False)
df.to_excel('sleep.xlsx', index=False)

# Stress
df = pd.DataFrame(stress)
df.to_csv('stress.csv', index=False)
df.to_excel('stress.xlsx', index=False)

# Day Stress Aggregated
df = pd.DataFrame(day_stress_aggregated)
df.to_csv('day_stress_aggregated.csv', index=False)
df.to_excel('day_stress_aggregated.xlsx', index=False)

# Respiration
df = pd.DataFrame(respiration)
df.to_csv('respiration.csv', index=False)
df.to_excel('respiration.xlsx', index=False)

# Race Prediction
df = pd.DataFrame(race_prediction)
df.to_csv('race_prediction.csv', index=False)
df.to_excel('race_prediction.xlsx', index=False)

# Spo2
df = pd.DataFrame(spo2)
df.to_csv('spo2.csv', index=False)
df.to_excel('spo2.xlsx', index=False)

# Max Metrics
df = pd.DataFrame(max_metrics)
df.to_csv('max_metrics.csv', index=False)
df.to_excel('max_metrics.xlsx', index=False)

# Personal Record
df = pd.DataFrame(personal_record)
df.to_csv('personal_record.csv', index=False)
df.to_excel('personal_record.xlsx', index=False)

# Earned Badges
df = pd.DataFrame(earned_badges)
df.to_csv('earned_badges.csv', index=False)
df.to_excel('earned_badges.xlsx', index=False)

# Adhoc Challenges
df = pd.DataFrame(adhoc_challenges)
df.to_csv('adhoc_challenges.csv', index=False)
df.to_excel('adhoc_challenges.xlsx', index=False)

# Available Badges
df = pd.DataFrame(available_badges)
df.to_csv('available_badges.csv', index=False)
df.to_excel('available_badges.xlsx', index=False)

# Available Badge Challenges
df = pd.DataFrame(available_badge_challenges)
df.to_csv('available_badge_challenges.csv', index=False)
df.to_excel('available_badge_challenges.xlsx', index=False)

# Badge Challenges
df = pd.DataFrame(badge_challenges)
df.to_csv('badge_challenges.csv', index=False)
df.to_excel('badge_challenges.xlsx', index=False)

# Non Completed Badge Challenges
df = pd.DataFrame(non_completed_badge_challenges)
df.to_csv('non_completed_badge_challenges.csv', index=False)
df.to_excel('non_completed_badge_challenges.xlsx', index=False)

# Activities
df = pd.DataFrame(activities)
df.to_csv('activities.csv', index=False)
df.to_excel('activities.xlsx', index=False)

# Activities Date
df = pd.DataFrame(activities_date)
df.to_csv('activities_date.csv', index=False)
df.to_excel('activities_date.xlsx', index=False)

# Activities Fordate Aggregated
df = pd.DataFrame(activities_fordate_aggregated)
df.to_csv('activities_fordate_aggregated.csv', index=False)
df.to_excel('activities_fordate_aggregated.xlsx', index=False)

# Devices
df = pd.DataFrame(devices)
df.to_csv('devices.csv', index=False)
df.to_excel('devices.xlsx', index=False)

# Device Last Used
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([device_last_used])
df.to_csv('device_last_used.csv', index=False)
df.to_excel('device_last_used.xlsx', index=False)

# Device Settings
df = pd.DataFrame(device_settings)
df.to_csv('device_settings.csv', index=False)
df.to_excel('device_settings.xlsx', index=False)

# Device Alarms
df = pd.DataFrame(device_alarms)
df.to_csv('device_alarms.csv', index=False)
df.to_excel('device_alarms.xlsx', index=False)

# Active Goals
df = pd.DataFrame(active_goals)
df.to_csv('active_goals.csv', index=False)
df.to_excel('active_goals.xlsx', index=False)

# Future Goals
df = pd.DataFrame(future_goals)
df.to_csv('future_goals.csv', index=False)
df.to_excel('future_goals.xlsx', index=False)

# Past Goals
df = pd.DataFrame(past_goals)
df.to_csv('past_goals.csv', index=False)
df.to_excel('past_goals.xlsx', index=False)

# HRV
df = pd.DataFrame(hrv)
df.to_csv('hrv.csv', index=False)
df.to_excel('hrv.xlsx', index=False)

# Weigh Ins
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([weigh_ins])
df.to_csv('weigh_ins.csv', index=False)
df.to_excel('weigh_ins.xlsx', index=False)

# Weigh Ins Daily
df = pd.DataFrame(weigh_ins_daily)
df.to_csv('weigh_ins_daily.csv', index=False)
df.to_excel('weigh_ins_daily.xlsx', index=False)

# Hill Score
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([hill_score])
df.to_csv('hill_score.csv', index=False)
df.to_excel('hill_score.xlsx', index=False)

# Endurance Score
# ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
# Fix by converting to a list of dicts
df = pd.DataFrame([endurance_score])
df.to_csv('endurance_score.csv', index=False)
df.to_excel('endurance_score.xlsx', index=False)

# Inprogress Virtual Challenges
df = pd.DataFrame(inprogress_virtual_challenges)
df.to_csv('inprogress_virtual_challenges.csv', index=False)
df.to_excel('inprogress_virtual_challenges.xlsx', index=False)



# 5. Visualization

We've extracted lots of data, but what does it look like?

In this section, we will be visualizing our all kinds of data in a simple, customizable plot! This plot is intended to provide a starter example for plotting, whereas later examples emphasize deep control and aesthetics.

#### For following Data Fields, situation is like either there is no real data which can be used as example to generate randomized data OR the data field is not suitable for Visualization :
- Body Composition - (No Data VALUES)
- Body Composition Aggregated - (No Data VALUES)
- Blood Pressure - (No Data VALUES)
- Training Status - (No Data)
- Resting Heart Rate (*Complex but **TO BE IMPLEMENTED** later after pre-processing*)
- SPO2 (No data)
- Personal Record (Not Suitable)
- Earned Badges (Not Suitable)
- Adhoc Challenges (No Data)
- Available Badges (Not Suitable)
- Available Badge Challenges (No Data)
- Badge Challenges (No Data)
- Non Completed Badge Challenges (Not Suitable)
- Devices (Not Suitable)
- Device Last Used (Not Suitable)
- Device Settings (Not Suitable)
- Device Alarms (Not Suitable)
- Active Goals (No Data)
- Future Goals (No Data)
- Past Goals (No Data)
- Weign-ins (No Data VALUES)
- Weigh-ins-daily (No Data VALUES)
- Hill Score (No Data VALUES)
- Endurance Score (No Data VALUES)
- Race Prediction (No Data)
- Inprogress Virtual Challenges (No Data)

In [ ]:
from datetime import datetime, timedelta
from logging import exception
#@title Plots Variables { display-mode: "form" }
# "body_composition", "Blood Pressure", "personal_record"

feature = "stats" #@param ["stats",  "stats_and_body_aggregated", "steps", "body_battery", "hr", "training_readiness", "floors", "hydration", "sleep", "stress","day_stress_aggregated","respiration", "max_metrics", "activities", "activities_date", "activities_fordate_aggregated", "hrv" ]
visual_start_date = "2022-03-04" #@param {type:"date"}
# visual_end_date = "2022-04-04" #@param {type: "date"}
time_interval = "One Day" #@param ["One Day", "One Week (7 Days)", "One Month (30 Days)", "One Year (365 Days)", "Entire Time Period"]
smoothness = 0 #@param {type:"slider", min:0, max:1, step:0.01}
smooth_plot = False #@param {type:"boolean"}

# Valid Interval Check
dates = device.get_data("dates", params=params)
visual_start_date = datetime.strptime(visual_start_date, '%Y-%m-%d')
if visual_start_date + timedelta(days=365) > dates[-1] and time_interval == "One Year (365 Days)":
  raise Exception("Not enough data. Please select appropriate time interval.")
elif visual_start_date + timedelta(days=30) > dates[-1] and time_interval == "One Month (30 Days)":
  raise Exception("Not enough data. Please select appropriate time interval.")
elif visual_start_date + timedelta(days=7) > dates[-1] and time_interval == "One Week (7 Days)":
  raise Exception("Not enough data. Please select appropriate time interval.")
elif visual_start_date + timedelta(days=1) > dates[-1] and time_interval == "One Day":
  raise Exception("Not enough data!")

start_date_index = dates.index(visual_start_date)
end_date_index = 0

if time_interval == "One Day":
    end_date_index = dates.index(visual_start_date + timedelta(days=1))
elif time_interval == "One Week (7 Days)":
    end_date_index = dates.index(visual_start_date + timedelta(days=7))
elif time_interval == "One Month (30 Days)":
    end_date_index = dates.index(visual_start_date + timedelta(days=30))
elif time_interval == "One Year (365 Days)":
    end_date_index = dates.index(visual_start_date + timedelta(days=365))
elif time_interval == "Entire Time Period":
    end_date_index = len(dates) - 1


In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


if feature == "stats":
    stats_data = stats[start_date_index:end_date_index+1]
    pass
elif feature == "body_composition":
    # print("NA")
    pass
elif feature == "body_composition_aggregated":
    # body_composition_aggregated_data = body_composition_aggregated[start_date_index:end_date_index+1]
    pass
elif feature == "stats_and_body_aggregated":
    # stats_and_body_aggregated_data = stats_body_aggregated[start_date_index:end_date_index+1]
    pass
elif feature == "steps":
    # steps_data = steps[start_date_index:+1] # 15 mins
    pass
elif feature == "body_battery":
    # body_battery_data = body_battery[start_date_index:end_date_index+1]
    pass
elif feature == "hr":
    # hr_data = hr[start_date_index:end_date_index+1]
    pass
elif feature == "training_readiness":
    # training_readiness_data = training_readiness[start_date_index:end_date_index+1]
    pass
elif feature == "blood_pressure":
    # print("NA")
    pass
elif feature == "floors":
    # floors_data = floors[start_date_index:end_date_index+1] # 15 mins
    pass
elif feature == "training_status":
    # training_status_data = training_status[start_date_index:end_date_index+1]
    # print("NO REAL DATA")
    pass
elif feature == "rhr":
    # print("COMPLICATED OBJECT DATA")
    pass
elif feature == "hydration":
    # hyrdation_data = hydration[start_date_index:end_date_index+1]
    pass
elif feature == "sleep":
    # sleep_data = sleep[start_date_index:end_date_index+1]
    pass
elif feature == "stress":
    # stress_data = stress[start_date_index:end_date_index+1]
    pass
elif feature == "day_stress_aggregated":
    # day_stress_aggregated_data = day_stress_aggregated[start_date_index:end_date_index+1]
    pass
elif feature == "respiration":
    # respiration_data = respiration[start_date_index:end_date_index+1]
    pass
elif feature == "spo2":
    # spo2_data = spo2[start_date_index:end_date_index+1]
    # print("NO REAL DATA")
    pass
elif feature == "max_metrics":
    # max_metrics_data = max_metrics[start_date_index:end_date_index+1]
    pass
elif feature == "personal_record":
    # print("NOT SUITABLE")
    pass
elif feature == "earned_badges":
    # print("NOT SUITABLE")
    pass
elif feature == "adhoc_challenges":
    # print("NOT SUITABLE")
    pass
elif feature == "available_badges":
    # print("NOT SUITABLE")
    pass
elif feature == "available_badge_challenges":
    # print("NOT SUITABLE")
    pass
elif feature == "badge_challenges":
    # print("NON_SUITABLE")
    pass
elif feature == "non_completed_badge_challenges":
    # print("NON_SUITABLE")
    pass
elif feature == "activities":
    # activities_data = activities[start_date_index:end_date_index+1]
    # print("RANDOM FOR REAL DATA BUT CONSISTENT FOR SYNTH DATA")
    pass
elif feature == "activities_date":
    # activities_date_data = activities_date[start_date_index:end_date_index+1]
    # print("RANDOM FOR REAL DATA BUT CONSISTENT FOR SYNTH DATA")
    pass
elif feature == "activities_fordate_aggregated":
    # activities_fordate_aggregated_data = activities_fordate_aggregated[start_date_index:end_date_index+1]
    pass
elif feature == "devices":
    # print("NOT SUITABLE")
    pass
elif feature == "device_last_used":
    # print("NOT SUITABLE")
    pass
elif feature == "device_settings":
    # print("NOT_SUITABLE")
    pass
elif feature == "device_alarms":
    # print("NO DATA")
    pass
elif feature == "active_goals":
    # print("NO DATA")
    pass
elif feature == "future_goals":
    # print("NO DATA")
    pass
elif feature == "past_goals":
    # print("NO DATA")
    pass
elif feature == "hrv":
    # hrv_data = hrv[start_date_index:end_date_index+1]
    pass
elif feature == "weigh_ins":
    # print("NA")
    pass
elif feature == "weigh_ins_daily":
    # weigh_ins_daily_data = weigh_ins_daily[start_date_index:end_date_index+1]
    # print("NO REAL DATA")
    pass
elif feature == "hill_score":
    # print("NA")
    pass
elif feature == "endurance_score":
    # print("NA")
    pass
elif feature == "inprogress_virtual_challenges":
    # print("NA")
    pass
